In [1]:
!pip install music21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 55.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-music/Caught Up In You.mid


In [106]:
import glob
import pretty_midi

def get_notes_and_instruments_from_midi(midifiles_path):
    all_notes = []
    all_instruments = []

    for file in glob.glob(midifiles_path + "/*.mid"):
        midi_data = pretty_midi.PrettyMIDI(file)
        print(f"Parsing {file}")

    for instrument in midi_data.instruments:
        for note in instrument.notes:
            all_notes.append(str(note.pitch))
            all_instruments.append(instrument.name)

    return all_notes, all_instruments

# Example usage
notes, instruments = get_notes_and_instruments_from_midi("/kaggle/input/test-music")
print(f"Collected {len(notes)} notes/chords.")

Parsing /kaggle/input/test-music/Caught Up In You.mid
Collected 4329 notes/chords.


In [41]:
df = pd.DataFrame({'Note': notes, 'Instrument': instruments})

In [32]:
from collections import defaultdict
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Encode the notes
encoder = LabelEncoder()
encoded_notes = encoder.fit_transform(notes)
n_vocab = len(set(encoded_notes))

sequence_length = 100  # You can experiment with this

# Create input sequences and the corresponding outputs
network_input = []
network_output = []

for i in range(len(encoded_notes) - sequence_length):
    seq_in = encoded_notes[i:i + sequence_length]
    seq_out = encoded_notes[i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

n_patterns = len(network_input)
print(f"Total patterns: {n_patterns}")

# Reshape and normalize input
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

# One-hot encode the output
network_output = to_categorical(network_output, num_classes=n_vocab)

Total patterns: 4229


In [47]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

def create_model(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
#     model.add(Dropout(0.1))
    model.add(LSTM(512, return_sequences=True))
#     model.add(Dropout(0.1))
    model.add(LSTM(512))
    model.add(Dense(256))
#     model.add(Dropout(0.1))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

model = create_model(network_input, n_vocab)
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100, 512)       │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 28)             │         7,196 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 28)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,389,596 (20.56 MB)

 Trainable params: 5,389,596 (20.56 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [75]:
# Train the model
model.fit(network_input, network_output, epochs=100, batch_size=64)

Epoch 1/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0319
Epoch 2/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0380
Epoch 3/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0356
Epoch 4/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.0258
Epoch 5/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.0364
Epoch 6/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.0252
Epoch 7/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 0.0424
Epoch 8/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 0.0506
Epoch 9/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 0.0231
Epoch 10/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - loss: 0.0307
Epoch 11/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - loss: 0.0169
Epoch 12/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - loss: 0.0179
Epoch 13/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - loss: 0.0299
Epoch 14/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - loss: 0.0412
Epoch 15/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - lo

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Gather Data
# Assume 'data.csv' contains the note and instrument data.
data = pd.DataFrame({'note': notes, 'instrument': instruments})

# Step 2: Preprocess Data
# Convert notes and instruments into numerical representations
label_encoder_notes = LabelEncoder()
label_encoder_instruments = LabelEncoder()
data['note'] = label_encoder_notes.fit_transform(data['note'])
data['instrument'] = label_encoder_instruments.fit_transform(data['instrument'])

# Define features and target variable
X = data[['note']]
print(data['note'])
y = data['instrument']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Select a Classification Algorithm
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 4: Train the Model
rf_model.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Step 6: Display Results
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Classification Report:')
print(report)

0       24
1       23
2       24
3       22
4       22
        ..
4324     2
4325     7
4326     7
4327     4
4328     7
Name: note, Length: 4329, dtype: int64
Accuracy: 74.36%
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       241
           1       0.71      0.06      0.11        88
           2       0.94      0.97      0.95       218
           3       0.58      0.81      0.68       232
           4       0.52      0.33      0.41        87

    accuracy                           0.74       866
   macro avg       0.72      0.61      0.60       866
weighted avg       0.75      0.74      0.71       866



In [27]:
!pip install pretty_midi

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592292 sha256=b2fe02a46348346148ab9b6f2388bb3c3d36056ca9f95f1a6edb0f89123118e7
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 require

In [101]:
# Generate music
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(encoder.classes_))
pattern = network_input[start]

pattern = [int_to_note[int(value * n_vocab)] for value in pattern]
prediction_output = []


num_notes_to_generate=100

for note_index in range(num_notes_to_generate):
    input_seq = [encoder.transform([n])[0] for n in pattern]
    input_seq = np.reshape(input_seq, (1, len(input_seq), 1))
    input_seq = input_seq / float(n_vocab)

    # Predict
    prediction = model.predict(input_seq, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    # Append to the pattern
    pattern.append(result)
    pattern = pattern[1:]


preds = pd.DataFrame(prediction_output, columns = ['note'])

preds['note'] = label_encoder_notes.transform(preds['note'])

predicted_instruments = rf_model.predict(preds[['note']])
predicted_instruments = label_encoder_instruments.inverse_transform(predicted_instruments)

# Create MIDI data
midi = pretty_midi.PrettyMIDI()
instrument_track = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program('Acoustic Grand Piano'))


for i, note_name in enumerate(prediction_output):
    # Adjust note duration (replace with your desired duration)
    note_duration = 0.3
    note_start = i * note_duration
    note_end = note_start + note_duration

    note = pretty_midi.Note(velocity=79, pitch=int(note_name), start=note_start, end=note_end)
    instrument_track.notes.append(note)

    # Assign instrument to note
    instrument_track.name = predicted_instruments[i]

    midi.instruments.append(instrument_track)

midi.write('generated_music.mid')

/tmp/ipykernel_36/2675531609.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pattern = [int_to_note[int(value * n_vocab)] for value in pattern]
